In [ ]:
mutable struct Agent
    skill :: Float64 
    wage :: Float64
    employed :: Int
    UI :: Float64   #when determine the range of UI, remember that UI can also be 0 besides the original range
    laidoff :: Int
    search :: Float64
    Pi_s :: Float64
end

Agent(employed=0) = Agent(rand(ls.h_grid),0,employed,0,0,0,0)

Agent()

function new_choice!(ag, ls)
    if ag.employed == 1
        agwi = searchsortedfirst(ls.w_grid, ag.wage)
        aghi = searchsortedfirst(ls.h_grid, ag.skill)
        ag.employed = ls.policy_v[agwi,aghi]
        if ag.employed == 0
            ag.laidoff = 1
            ag.UI = 0
        end
    end
end

function welcome_job!(ag, ls)
    ag.wage = rand(ls.w_grid)
    agwi = searchsortedfirst(ls.w_grid, ag.wage)
    aghi = searchsortedfirst(ls.h_grid, ag.skill)
    
    if ag.wage * ag.skill < 0.7 * ag.UI
        agui = searchsortedfirst(ls.I_grid,ag.UI)
        ag.employed = ls.V[agwi,aghi] > ls.Vb[agui,aghi] ? 1 : 0
        if ag.employed == 0
            ag.wage = 0
        else
            ag.search = 0
            ag.Pi_s = 0
        end
    else
        ag.employed = ls.policy_v[agwi,aghi]
        if ag.employed == 0
            ag.wage = 0
            ag.UI = 0
        else
            ag.search = 0
            ag.Pi_s = 0
        end
    end
end

function search_hard!(ag, ls)
    if ag.UI > 0 
        aghi = searchsortedfirst(ls.h_grid, ag.skill)
        agui = searchsortedfirst(ls.I_grid, ag.UI)
        ag.search = ls.policy_vb[agui,aghi]
        ag.Pi_s = ag.search^0.3
    else
        aghi = searchsortedfirst(ls.h_grid, ag.skill)
        ag.search = ls.policy_v0[aghi]
        ag.Pi_s = ag.search^0.3
    end   
end

num_agents = 10000
sepa_rate = 0.009
agent_indices = collect(1:num_agents)
agents = [Agent() for i=1:num_agents]
sim_length = 1500
unemploymentrate = zeros(sim_length)
changedate = 1000

for t in 1:sim_length
    
    if t % 20 == 0
        println("date = $t")
    end
    
    if t == 1
        for agent in agents
            agent.skill = rand(ls.h_grid)
            agent.employed = rand([1,0])
            if agent.employed == 1
                agent.wage = rand(ls.w_grid)
            else
                agent.search = rand(ls.s0_grid)
                agent.Pi_s = agent.search^0.3
            end
        end
    end
    
    if t == changedate              
        sepa_rate = 0.18
        for agent in agents
            if agent.employed == 0
                agent.skill = 1
            end
        end
    end
    
    if t == changedate + 1            
        sepa_rate = 0.009
    end
                
    for agent in agents
        agent.laidoff = 0
    end
    
    employedworker = []
    for agent in agents
        if agent.employed == 1
            agent.UI = 0
            push!(employedworker, agent)
        end
    end
    
    eworker_indices = collect(1:length(employedworker))
    sep_num = round(Int, length(employedworker) * sepa_rate)
    shuffle!(eworker_indices)
    separation_list = eworker_indices[1:sep_num]
    
    for agent in employedworker[separation_list]
        agent.employed = 0
        agent.laidoff = 1
        agent.UI = ls.I_grid[Int(ceil(agent.wage * agent.skill / (2/15)))]
        if t == changedate
            agent.skill = 1
        end
    end
    
    stillemployedworker = []
    for agent in agents
        if agent.employed == 1
            push!(stillemployedworker, agent)
        end
    end
    
    stilleworker_indices = collect(1:length(stillemployedworker))
    shuffle!(stilleworker_indices)
    upperskill_num = round(Int, length(stillemployedworker) * 0.1)
    upperskill_list = stilleworker_indices[1:upperskill_num]
    
    for agent in stillemployedworker[upperskill_list]
        if agent.skill == ls.h_grid[21]
            agent.skill = ls.h_grid[21]
        else
            aghi = searchsortedfirst(ls.h_grid, agent.skill)
            agent.skill = ls.h_grid[aghi+1]
        end
    end
    
    for agent in stillemployedworker
        new_choice!(agent,ls)
    end
    
    nojobguys = []
    for agent in agents
        if agent.employed == 0 && agent.laidoff == 0
            push!(nojobguys, agent)
        end
    end
    
    if length(nojobguys) > 0
        
        nojob_indices = collect(1:length(nojobguys))
        lowerskill_num = round(Int, length(nojobguys) * 0.2)
        shuffle!(nojob_indices)
        lowerskill_list = nojob_indices[1:lowerskill_num]
        
        for agent in nojobguys[lowerskill_list]
            if agent.skill == ls.h_grid[1]
                agent.skill = ls.h_grid[1]
            else
                aghi = searchsortedfirst(ls.h_grid, agent.skill)
                agent.skill = ls.h_grid[aghi-1]
            end
        end
    
    #In this model, the skill of laidoff workers has not been changed
    
        sum_pis = 0
        for agent in nojobguys
            sum_pis = sum_pis + agent.Pi_s
        end
    
        E_Pis = sum_pis / length(nojobguys)
        offer_num = round(Int, length(nojobguys) * E_Pis)
        shuffle!(nojob_indices)
        offer_list = nojob_indices[1:offer_num]
        
        for agent in nojobguys[offer_list]
            welcome_job!(agent, ls)
        end
    end
    
    
    newlife_num = round(Int, num_agents * 0.0009)
    shuffle!(agent_indices)
    newlife_list = agent_indices[1:newlife_num]
    
    for agent in agents[newlife_list]
        agent.employed = 0
        agent.UI = 0
        agent.wage = 0
        agent.laidoff = 0
        agent.skill = rand(ls.h_grid)
    end
    
    stillnojob = []
    for agent in agents
        if agent.employed == 0
            push!(stillnojob, agent)
        end
    end
    
    for agent in stillnojob
        search_hard!(agent, ls)
    end
    
    employment = Int[agent.employed for agent in agents]
    unemploymentrate[t] = 1 - mean(employment)
end

plot(unemploymentrate, linewidth = 2, label = "unemployment rate")
vline!([changedate], color = :red, label = "transient shock")

unemploymentrate